In [56]:
import piexif
from PIL import Image
import glob
from geopy.distance import vincenty
from itertools import groupby
import csv

In [57]:
def dms_to_dd(gps_data):
    
    ##GPS data is in Deg min sec to be converted in Degrees
    Degrees = float(gps_data[0][0]) / float(gps_data[0][1])

    Minutes = float(gps_data[1][0]) / float(gps_data[1][1])

    Seconds = float(gps_data[2][0]) / float(gps_data[2][1])

    return Degrees + (Minutes / 60.0) + (Seconds / 3600.0)

    
    

In [58]:
def calculate_distance_on_earth(strt_lat_dec_deg,strt_long_dec_deg,stp_lat_dec_deg,stp_long_dec_deg):
    #Enter the start and stop lattitude and Longitude in Decimal Degrees 
    start=(strt_lat_dec_deg,strt_long_dec_deg)
    stop=(stp_lat_dec_deg,stp_long_dec_deg)
    dist_in_meters=vincenty(start,stop).meters
    return dist_in_meters 



In [59]:
def get_gps_data_from_images():
    imagesData = {}

    
    for img in glob.glob('/home/vicky/Desktop/Skylark_Drones/images/*.JPG'):
        
        img_dict = piexif.load(img)
        GPS_data =img_dict.get('GPS')
        try:
            GPS_latitude  = GPS_data[2]
            GPS_longitude = GPS_data[4]
        except:
            print("Key 2 is not found")
        latitude=dms_to_dd(GPS_latitude)
        longitude=dms_to_dd(GPS_longitude)
        imagesData[img[20:]]= [longitude, latitude]
    return imagesData

In [60]:
def get_images(images_data, longitude, latitude, req_distance):
    """
    Retrieves images lying within the radius of a 
    frame location using haversine formulae.
    """

    image_list = []
    
    for image_name, image_data in images_data.items():
        longitude1, latitude1= map(float, image_data)

        distance = calculate_distance_on_earth(longitude, latitude, longitude1, latitude1)

          
        if distance < req_distance:
            image_list.append(image_name)

    return image_list


In [61]:
def check_frames(frames, images_data):
    """Get the images from 35meters"""
    for frame in frames:
        lon2, lat2, alt2 = frame[2].strip('\n').split(',')
        image_list = get_images(images_data, lon2, lat2, 35)
        csv_file(frame[1].split('-->')[0], image_list)


In [62]:
def srt():
       
    with open('/home/vicky/Desktop/Skylark_Drones/videos/DJI_0301.SRT') as file:
        results = [list(f)
               for t, f in groupby(file, lambda x: bool(x.strip())) if t]

    return results

In [63]:
def csv_file(frame_no, image_list):
    
    with open('/home/vicky/Desktop/Skylark_Drones/video.csv', 'a') as outfile:
        outfile.write(frame_no.replace(',', '.'))
        outfile.write(',')
        outfile.write(':'.join(image_list))
        outfile.write('\n')


In [69]:
frames=srt()
images_data=get_gps_data_from_images()
check_frames(frames, images_data)

Key 2 is not found
Key 2 is not found
Key 2 is not found
Key 2 is not found


In [67]:
def check_poi(filename, images_data):
    """
    Adds all images lying 50m near to Point of interests. 
    """

    with open(filename, 'r') as rfile, open('/home/vicky/Desktop/Skylark_Drones/spread_sheet.csv', 'w') as wfile:

        reader = csv.DictReader(rfile)

        fieldnames = ['asset_name', 'longitude', 'latitude', 'image_names']
        writer = csv.DictWriter(wfile, fieldnames=fieldnames)

        writer.writeheader()

        for row in reader:
            lon2 = row['longitude']
            lat2 = row['latitude']
            image_list = get_images(images_data, lon2, lat2, 50)

            row['image_names'] = ':'.join(image_list)
            writer.writerow(row)



In [68]:
check_poi('/home/vicky/Desktop/Skylark_Drones/assets.csv', images_data)

    